In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import utilities as ut

In [2]:
df  = pd.read_csv("GBPT/edges.csv")
#Extracting London PT
node_df = pd.read_csv("GBPT/nodes.csv")
#Creating the london df
# london_nodes_df = node_df.loc[node_df["zone"]==490]
# london_nodes = list(london_nodes_df["node"])
# ori_node = [];des_node=[];ori_layer=[];des_layer =[]
# for index,rows in df.iterrows():
#     if rows["ori_node"] in london_nodes:
#         ori_node.append(rows["ori_node"])
#         des_node.append(rows["des_node"])
#         ori_layer.append(rows["ori_layer"])
#         des_layer.append(rows["des_layer"])
        

# london_df = pd.DataFrame({"ori_node":ori_node,
#                           "des_node":des_node,
#                           "ori_layer":ori_layer,
#                           "des_layer":des_layer
#                           })
# london_df.to_csv("GBPT/processed/london_df.csv",index=False)

rail_df = df.loc[df["ori_layer"] == 2]
rail_df = rail_df.loc[rail_df["des_layer"] ==2]
source = rail_df["ori_node"]
target = rail_df["des_node"]
graph_df = pd.DataFrame(data={"source":source,"target":target})
G = ut.df_to_network(graph_df)
G.remove_edges_from(nx.selfloop_edges(G))
G = ut.gcc(G)
graph_df = ut.network_to_df(G)
graph_df.to_csv("real_networks/processed/GBPT_train.csv",index=False)

In [3]:
london_df = pd.read_csv("GBPT/processed/london_df.csv")
london_bus = london_df.loc[london_df["ori_layer"]==5]
london_bus = london_bus.loc[london_bus["des_layer"]==5]
source = london_bus["ori_node"];target = london_bus["des_node"]
london_bus = pd.DataFrame(data = {"source":source,"target":target})
G = ut.df_to_network(london_bus)
G.remove_edges_from(nx.selfloop_edges(G))
G = ut.gcc(G)
graph_df = ut.network_to_df(G)
graph_df.to_csv("bus/processed/london.csv",index=False)

In [4]:
#Beijing buses process

df = pd.read_csv("bus/raw/beijing.csv")
stops = set(df["站点名称"])
lines = set(df["线路名称"])
lines_id = np.linspace(1,len(lines),len(lines))
lines_id = [int(item) for item in lines_id]
stops_id = np.linspace(1,len(stops),len(stops))
stops_id = [int(item) for item in stops_id]

line_dict = dict(zip(lines_id,lines))
stop_dict = dict(zip(stops_id,stops))
stops_id_conversion =[]
for stop in df["站点名称"]:
    stops_id_conversion.append(list(stop_dict.values()).index(stop))
stops_id_conversion = [stop+1 for stop in stops_id_conversion]
df["Stop_ID"] = stops_id_conversion



lines_list_seperate = []
for item in list(lines):
    temp_list = df.loc[df["线路名称"]==item]
    lines_list_seperate.append(temp_list)
    
G = nx.Graph()
G.add_nodes_from(stops_id)

for lines in lines_list_seperate:
    for i in range(len(lines)):
        for first, second in zip(lines["Stop_ID"], lines["Stop_ID"][1:]):
            G.add_edge(first,second)

#Remove self-loops
G.remove_edges_from(nx.selfloop_edges(G))
G = ut.gcc(G)
edges = list(G.edges())
source = [item[0] for item in edges]
target = [item[1] for item in edges]
data = {"source" : source,"target" : target}
processed_beijing_bus = pd.DataFrame(data = data)
processed_beijing_bus.to_csv("bus/processed/beijing.csv",index = False)

In [5]:
# Other cities
cities = ["paris","berlin","sydney","detroit"]
load_path = ["bus/raw/" + item + ".csv" for item in cities]
dfs = [pd.read_csv(item,sep=';'  , engine='python') for item in load_path]
normalised_df = []
for df in dfs:
    source = df["from_stop_I"]
    target = df["to_stop_I"]
    temp_df = pd.DataFrame(data={"source":source,"target":target})
    normalised_df.append(temp_df)
graphs = [ut.df_to_network(item) for item in normalised_df]
for item in graphs:
    item.remove_edges_from(nx.selfloop_edges(item))
graphs = [ut.gcc(g) for g in graphs]
normalised_df = [ut.network_to_df(item) for item in graphs]
for i in range(len(cities)):
    normalised_df[i].to_csv("bus/processed/"+cities[i]+".csv",index=False)

In [6]:
def remove_degree_2(G1):
    G= G1.copy()
    flag = 1
    while flag == 1:
        node_len = len(G.nodes)
        remove_list = []
        for node in list(G.nodes):
            if G.degree(node) == 2:
                remove_list.append(node)
        
        for node in remove_list:
            neighbors = tuple(G.neighbors(node))
            if len(neighbors) == 2:
                G.remove_node(node)
                G.add_edge(neighbors[0],neighbors[1])
        if node_len == len(G.nodes):
            flag =0
    return G

In [7]:
# Modified buses
cities = ["london","paris","berlin","sydney","detroit","beijing"]
load_path = ["bus/processed/"+item+".csv" for item in cities]
dfs = [pd.read_csv(item) for item in load_path]
graphs = [ut.df_to_network(item) for item in dfs]
graphs = [remove_degree_2(item) for item in graphs]
modified_dfs = [ut.network_to_df(item) for item in graphs]
for i in range(len(cities)):
    modified_dfs[i].to_csv("bus/modified/m_"+cities[i]+".csv",index=False)

In [8]:
# Other real networks
names = ["dolphins","pdzbase","hamsterster","Roget","flight"]
load_path = ["real_networks/raw/"+item+".csv" for item in names]
dfs = [pd.read_csv(item) for item in load_path]
graphs = [ut.df_to_network(item) for item in dfs]
for item in graphs:
    item.remove_edges_from(nx.selfloop_edges(item))
graphs = [ut.gcc(g) for g in graphs]
dfs = [ut.network_to_df(item) for item in graphs]
for i in range(len(names)):
    dfs[i].to_csv("real_networks/processed/"+names[i]+".csv",index=False)